In [195]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score, GridSearchCV

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
# pd.set_option('display.max_rows', None)


#### EDA

In [196]:
# Dataset
dataset = pd.read_csv('Walmart_Store_sales.csv')
dataset.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,18-02-2011,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,25-03-2011,1807545.43,0.0,42.38,3.435,128.616064,7.470
2,17.0,27-07-2012,NaN,0.0,NaN,NaN,130.719581,5.936
3,11.0,NaN,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,28-05-2010,1644470.66,0.0,78.89,2.759,212.412888,7.092


In [197]:
dataset.head(100)

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,18-02-2011,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,25-03-2011,1807545.43,0.0,42.38,3.435,128.616064,7.470
2,17.0,27-07-2012,NaN,0.0,NaN,NaN,130.719581,5.936
3,11.0,NaN,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,28-05-2010,1644470.66,0.0,78.89,2.759,212.412888,7.092
...,...,...,...,...,...,...,...,...
95,1.0,14-05-2010,1494251.50,0.0,74.78,2.854,210.337426,7.808
96,8.0,12-03-2010,860336.16,0.0,49.76,NaN,214.749245,6.299
97,5.0,30-04-2010,298697.84,0.0,67.53,NaN,210.939388,NaN
98,10.0,25-06-2010,1768172.31,0.0,90.32,NaN,126.126600,9.524


In [198]:
# Number of rows and columns
print("Number of rows : ", dataset.shape[0])
print("Number of columns : ", dataset.shape[1])

Number of rows :  150
Number of columns :  8


Remarque : On observe d'ores et déjà que le dataset est très petit, seulement 150 observations sans aucun preprocessing !

In [199]:
# Columns type
dataset.dtypes

Store           float64
Date             object
Weekly_Sales    float64
Holiday_Flag    float64
Temperature     float64
Fuel_Price      float64
CPI             float64
Unemployment    float64
dtype: object

Remarque : Toutes les colonnes sont de type float, exceptée la colonne "Date" qui est de type object, et qui sera donc l'objet d'un traitement particulier

In [200]:
# Dataset description
dataset.describe(include='all')

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,150.000000,132,1.360000e+02,138.000000,132.000000,136.000000,138.000000,135.000000
unique,NaN,85,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,19-10-2012,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN
mean,9.866667,NaN,1.249536e+06,0.079710,61.398106,3.320853,179.898509,7.598430
std,6.231191,NaN,6.474630e+05,0.271831,18.378901,0.478149,40.274956,1.577173
min,1.000000,NaN,2.689290e+05,0.000000,18.790000,2.514000,126.111903,5.143000
25%,4.000000,NaN,6.050757e+05,0.000000,45.587500,2.852250,131.970831,6.597500
50%,9.000000,NaN,1.261424e+06,0.000000,62.985000,3.451000,197.908893,7.470000
75%,15.750000,NaN,1.806386e+06,0.000000,76.345000,3.706250,214.934616,8.150000


In [201]:
# Missing values
(100 * dataset.isnull().sum() / dataset.shape[0])

Store            0.000000
Date            12.000000
Weekly_Sales     9.333333
Holiday_Flag     8.000000
Temperature     12.000000
Fuel_Price       9.333333
CPI              8.000000
Unemployment    10.000000
dtype: float64

Conclusions :
- Très peu d'observations : On a donc une très grande probabilité d'overfitting, qu'il faudra traiter
- Il faudra traiter les valeurs manquantes présentes en particulier celles de la valeur cible (weekly_sales)
- Un traitement particulier devra être effectué sur la colonne Date
- Il faudra traiter les outliers

In [202]:
# Suppression des lignes avec valeur target (weekly_sales) manquante
rows_to_keep = ~(dataset["Weekly_Sales"].isnull())
dataset = dataset.loc[rows_to_keep, :]
display(dataset)

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,18-02-2011,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,25-03-2011,1807545.43,0.0,42.38,3.435,128.616064,7.470
3,11.0,NaN,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,28-05-2010,1644470.66,0.0,78.89,2.759,212.412888,7.092
5,4.0,28-05-2010,1857533.70,0.0,NaN,2.756,126.160226,7.896
...,...,...,...,...,...,...,...,...
145,14.0,18-06-2010,2248645.59,0.0,72.62,2.780,182.442420,8.899
146,7.0,NaN,716388.81,NaN,20.74,2.778,NaN,NaN
147,17.0,11-06-2010,845252.21,0.0,57.14,2.841,126.111903,NaN
148,8.0,12-08-2011,856796.10,0.0,86.05,3.638,219.007525,NaN


In [203]:
# Split column Date into year, month, day
dataset["Date"] = pd.to_datetime(dataset["Date"], format="%d-%m-%Y")
dataset["Year"] = dataset["Date"].dt.year.astype('Int64')
dataset["Month"] = dataset["Date"].dt.month.astype('Int64')
dataset["Day"] = dataset["Date"].dt.day.astype('Int64')
dataset["Day_of_week"] = dataset["Date"].dt.day_of_week.astype("Int64")
dataset

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Day,Day_of_week
0,6.0,2011-02-18,1572117.54,NaN,59.61,3.045,214.777523,6.858,2011,2,18,4
1,13.0,2011-03-25,1807545.43,0.0,42.38,3.435,128.616064,7.470,2011,3,25,4
3,11.0,NaT,1244390.03,0.0,84.57,NaN,214.556497,7.346,<NA>,<NA>,<NA>,<NA>
4,6.0,2010-05-28,1644470.66,0.0,78.89,2.759,212.412888,7.092,2010,5,28,4
5,4.0,2010-05-28,1857533.70,0.0,NaN,2.756,126.160226,7.896,2010,5,28,4
...,...,...,...,...,...,...,...,...,...,...,...,...
145,14.0,2010-06-18,2248645.59,0.0,72.62,2.780,182.442420,8.899,2010,6,18,4
146,7.0,NaT,716388.81,NaN,20.74,2.778,NaN,NaN,<NA>,<NA>,<NA>,<NA>
147,17.0,2010-06-11,845252.21,0.0,57.14,2.841,126.111903,NaN,2010,6,11,4
148,8.0,2011-08-12,856796.10,0.0,86.05,3.638,219.007525,NaN,2011,8,12,4


In [204]:
# Drop lines containing outliers

# Temperature
print("Nombre de lignes avant suppression Temperature: ", dataset.shape[0])
rows_to_keep = (dataset["Temperature"].isnull()) | (dataset["Temperature"] < dataset["Temperature"].mean() + 3 * dataset["Temperature"].std()) & (dataset["Temperature"] > dataset["Temperature"].mean() - 3 * dataset["Temperature"].std())
dataset = dataset.loc[rows_to_keep, :]
print("Nombre de lignes après suppression Temperature: ", dataset.shape[0])
print()
# No outlier found for Temperature

# Fuel_Price
print("Nombre de lignes avant suppression Fuel_price: ", dataset.shape[0])
rows_to_keep = (dataset["Fuel_Price"].isnull()) | (dataset["Fuel_Price"] < dataset["Fuel_Price"].mean() + 3 * dataset["Fuel_Price"].std()) & (dataset["Fuel_Price"] > dataset["Fuel_Price"].mean() - 3 * dataset["Fuel_Price"].std())
dataset = dataset.loc[rows_to_keep, :]
print("Nombre de lignes après suppression Fuel_Price: ", dataset.shape[0])
print()
# No outlier found for Fuel_Price

# CPI
print("Nombre de lignes avant suppression CPI: ", dataset.shape[0])
rows_to_keep = (dataset["CPI"].isnull()) | (dataset["CPI"] < dataset["CPI"].mean() + 3 * dataset["CPI"].std()) & (dataset["CPI"] > dataset["CPI"].mean() - 3 * dataset["CPI"].std())
dataset = dataset.loc[rows_to_keep, :]
print("Nombre de lignes après suppression CPI: ", dataset.shape[0])
print()
# No outlier found for CPI

# Unemployment
print("Nombre de lignes avant suppression Unemployment: ", dataset.shape[0])
rows_to_keep = (dataset["Unemployment"].isnull()) | (dataset["Unemployment"] < dataset["Unemployment"].mean() + 3 * dataset["Unemployment"].std()) & (dataset["Unemployment"] > dataset["Unemployment"].mean() - 3 * dataset["Unemployment"].std())
dataset = dataset.loc[rows_to_keep, :]
print("Nombre de lignes après suppression Unemployment: ", dataset.shape[0])
# 5 lines deleted for Unemployment
print("5 lignes supprimées pour des outliers de la colonne Unemployment")

Nombre de lignes avant suppression Temperature:  136
Nombre de lignes après suppression Temperature:  136

Nombre de lignes avant suppression Fuel_price:  136
Nombre de lignes après suppression Fuel_Price:  136

Nombre de lignes avant suppression CPI:  136
Nombre de lignes après suppression CPI:  136

Nombre de lignes avant suppression Unemployment:  136
Nombre de lignes après suppression Unemployment:  131
5 lignes supprimées pour des outliers de la colonne Unemployment


#### Baseline model

Essayons de déterminer la corrélation entre la variable cible (weekly_sales) et les variables explicatives

In [205]:
fig = px.scatter_matrix(dataset)
fig.update_layout(
        title = go.layout.Title(text = "Bivariate analysis", x = 0.5), showlegend = False, 
            autosize=False, height=1500, width = 1500)
fig.show()

In [206]:
corr_matrix = dataset.corr().round(2)
import plotly.figure_factory as ff
fig = ff.create_annotated_heatmap(corr_matrix.values,
                                  x = corr_matrix.columns.tolist(),
                                  y = corr_matrix.index.tolist())


fig.show()

La Matrice de corrélation nous permet de voir qu'il n'y a pas de variables explicatives fortement corrélées avec la variable cible (weekly_sales). L'entrainement d'un modèle de régression linéaire avec une seule variable explicative a donc de fortes chances de ne pas donner de bons résultats.

Essayons tout de même d'entrainer un modèle Simple Linear Regression avec Unemployment comme seule variable explicative

In [207]:
target_name = "Weekly_Sales"
Y = dataset.loc[:, target_name]

X = dataset[['Unemployment']]

In [208]:
# Definition of train and test variables 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

In [209]:
numeric_features = ["Unemployment"]
numeric_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="mean")),
        ("scaler", StandardScaler())
    ]
)

In [210]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features)
    ]
)

In [211]:
# Preprocessings on train set
print('Performing preprocessings on train set...')
print(X_train.head())
X_train = preprocessor.fit_transform(X_train)
print("Done...")
print(X_train[0:5])
print()

# Preprocessings on test set
print("Performing preprocessings on test set...")
print(X_test.head())
X_test = preprocessor.transform(X_test)
print('Done...')
print(X_test[0:5])
print()

Performing preprocessings on train set...
     Unemployment
129         8.535
37          8.975
64          5.143
13          7.348
3           7.346
Done...
[[ 1.19624122]
 [ 1.65989105]
 [-2.37807746]
 [-0.05455957]
 [-0.05666707]]

Performing preprocessings on test set...
     Unemployment
46          6.496
91          5.936
147           NaN
6           7.658
48          7.962
Done...
[[-9.52354235e-01]
 [-1.54245402e+00]
 [ 9.35917663e-16]
 [ 2.72102811e-01]
 [ 5.92442693e-01]]



##### Train model

In [212]:
model = LinearRegression()
model.fit(X_train, Y_train)

LinearRegression()

##### Predictions

In [213]:
# Predictions on training set
print("Predictions on training set...")
Y_train_pred = model.predict(X_train)
print('Done !')
print(Y_train_pred[0:5])
print()

# Predictions on test set
print('Predictions on test set...')
Y_test_pred = model.predict(X_test)
print('Done !')
print(Y_test_pred[0:5])
print()



Predictions on training set...
Done !
[1332253.62527039 1370166.60028816 1039979.05422428 1229974.75857471
 1229802.42687008]

Predictions on test set...
Done !
[1156561.45240393 1108308.57510858 1234436.13307692 1256686.17279178
 1282880.59189497]



##### Evaluation

In [214]:
# Evaluation 
print("R² score on train set : ", r2_score(Y_train, Y_train_pred))
print("R² score on test set : ", r2_score(Y_test, Y_test_pred))

R² score on train set :  0.015815896232727833
R² score on test set :  0.04751621618116553


Le R² obtenu sur test comme sur train parle de lui-même. la performance de ce modèle avec "Unemployment" comme seule variable explicative est médiocre.

#### Multivariate linear regression

Nous allons maintenant utiliser l'ensemble des colonnes numériques comme variables explicatives

In [215]:
target_name = "Weekly_Sales"
Y = dataset.loc[:, target_name]

X = dataset.drop(target_name, axis=1)


- Target variable : Weekly_Sales
- Explanatory variables : Store, Date, Holiday_Flag, Temperature, Fuel_Price, CPI, Unemployment, Year, Month, Day, Day_of_week

In [216]:
dataset['Store'] = dataset["Store"].astype("int")
dataset.dtypes

Store                    int32
Date            datetime64[ns]
Weekly_Sales           float64
Holiday_Flag           float64
Temperature            float64
Fuel_Price             float64
CPI                    float64
Unemployment           float64
Year                     Int64
Month                    Int64
Day                      Int64
Day_of_week              Int64
dtype: object

In [217]:
# Definition of train and test variables 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

##### Preprocessing

In [218]:
# Numeric variables : Temperature, Fuel_Price, CPI, Unemployment, Year, Month, Day, Day_of_week
numeric_features_discrete = ["Year", "Month", "Day", "Day_of_week"]
numeric_transformer_discrete = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ]
)

numeric_features_continuous = ["Temperature", "Fuel_Price", "CPI", "Unemployment"]
numeric_transformer_continuous = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="mean")),
        ("scaler", StandardScaler())
    ]
)

In [219]:
# Categorical features : Store, Holiday_Flag
categorical_features = ["Store", "Holiday_Flag"]  
categorical_transformer = Pipeline(
    steps=[
        (
            "imputer",
            SimpleImputer(strategy="most_frequent"),
        ),  # missing values will be replaced by most frequent value
        (
            "encoder",
            OneHotEncoder(drop="first"),
        ),  # first column will be dropped to avoid creating correlations between features
    ]
)

In [220]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num_disc", numeric_transformer_discrete, numeric_features_discrete),
        ("num_cont", numeric_transformer_continuous, numeric_features_continuous),
        ("cat", categorical_transformer, categorical_features)
    ]
)

In [221]:
# Preprocessings on train set
print('Performing preprocessings on train set...')
print(X_train.head())
X_train = preprocessor.fit_transform(X_train)
print("Done...")
print(X_train[0:5])
print()

# Preprocessings on test set
print("Performing preprocessings on test set...")
print(X_test.head())
X_test = preprocessor.transform(X_test)
print('Done...')
print(X_test[0:5])
print()

Performing preprocessings on train set...
     Store       Date  Holiday_Flag  Temperature  Fuel_Price         CPI  \
129   18.0        NaT           0.0        73.25       3.840  138.185710   
37    18.0 2011-04-15           0.0          NaN       3.823  134.278467   
64     4.0 2011-12-16           0.0        36.44       3.149  129.898065   
13     1.0 2012-03-16           0.0        64.74       3.734  221.211813   
3     11.0        NaT           0.0        84.57         NaN  214.556497   

     Unemployment  Year  Month   Day  Day_of_week  
129         8.535  <NA>   <NA>  <NA>         <NA>  
37          8.975  2011      4    15            4  
64          5.143  2011     12    16            4  
13          7.348  2012      3    16            4  
3           7.346  <NA>   <NA>  <NA>         <NA>  
Done...
[[ 0.17418541 -0.07509806  0.05792844  0.          0.72491486  1.20098741
  -1.21963331  1.19624122  0.          0.          0.          0.
   0.          0.          0.          0.

##### Train model

In [222]:
model = LinearRegression()
model.fit(X_train, Y_train)

LinearRegression()

##### Predictions

In [223]:
# Predictions on training set
print("Predictions on training set...")
Y_train_pred = model.predict(X_train)
print('Done !')
print(Y_train_pred[0:5])
print()

# Predictions on test set
print('Predictions on test set...')
Y_test_pred = model.predict(X_test)
print('Done !')
print(Y_test_pred[0:5])
print()

Predictions on training set...
Done !
[1042225.59050862 1011237.11892931 2475216.80702306 1446934.32863927
 1392251.37619672]

Predictions on test set...
Done !
[ 261079.0636327   882759.6777613   734760.99622791  672708.18490017
 1480559.72973577]



##### Evaluation

In [224]:
print("R² score on train set : ", r2_score(Y_train, Y_train_pred))
print("R² score on test set : ", r2_score(Y_test, Y_test_pred))

R² score on train set :  0.976509899076633
R² score on test set :  0.9293617573996498


#### Conclusion : Les performances du modèle de régression linéaire sont bien plus élevées en ayant rajouté des variables explicatives.

On souhaiterait maintenant savoir quels sont les coefficients les plus importants

In [225]:
model.coef_

array([-3.49468921e+04,  5.76101987e+04, -2.48958794e+04, -8.14907253e-10,
       -5.92117617e+04, -1.74890901e+04,  1.19162896e+05, -6.43121948e+04,
        3.48167330e+05, -1.15391696e+06,  8.32006970e+05, -1.31828985e+06,
        9.02962204e+03, -9.29625797e+05, -7.67636289e+05, -1.14648912e+06,
        6.93750306e+05, -8.53533328e+04,  6.43981647e+05,  6.44409731e+05,
       -5.96906999e+05, -1.06914831e+06, -6.31573005e+05, -1.28195517e+05,
        1.51132327e+05,  4.50335901e+05,  6.36524199e+04])

In [227]:
column_names = []
for name, pipeline, features_list in preprocessor.transformers_:  # loop on pipelines
    if name in ["num_disc", "num_cont"]:  # numeric variables
        features = features_list  # get the column names
    elif isinstance(pipeline, str):  # if the pipeline is a string like 'passthrough' or 'drop'
        if pipeline == 'passthrough':
            features = features_list  # use original column names for passthrough
        else:
            features = []  # if 'drop', don't add any features
    else:  # if pipeline is for categorical variables 
        features = pipeline.named_steps['encoder'].get_feature_names_out(features_list)  
    column_names.extend(features)  # concatenate features names

print("Names of columns corresponding to each coefficient: ", column_names)

Names of columns corresponding to each coefficient:  ['Year', 'Month', 'Day', 'Day_of_week', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Store_2.0', 'Store_3.0', 'Store_4.0', 'Store_5.0', 'Store_6.0', 'Store_7.0', 'Store_8.0', 'Store_9.0', 'Store_10.0', 'Store_11.0', 'Store_13.0', 'Store_14.0', 'Store_15.0', 'Store_16.0', 'Store_17.0', 'Store_18.0', 'Store_19.0', 'Store_20.0', 'Holiday_Flag_1.0']


In [228]:
coefs = pd.DataFrame(index = column_names, data = model.coef_.transpose(), columns=["coefficients"])
coefs

,coefficients
Year,-3.494689e+04
Month,5.761020e+04
Day,-2.489588e+04
Day_of_week,-8.149073e-10
Temperature,-5.921176e+04
Fuel_Price,-1.748909e+04
CPI,1.191629e+05
Unemployment,-6.431219e+04
Store_2.0,3.481673e+05
Store_3.0,-1.153917e+06


In [229]:
feature_importance = abs(coefs).sort_values(by = 'coefficients')
feature_importance

,coefficients
Day_of_week,8.149073e-10
Store_6.0,9.029622e+03
Fuel_Price,1.748909e+04
Day,2.489588e+04
Year,3.494689e+04
Month,5.761020e+04
Temperature,5.921176e+04
Holiday_Flag_1.0,6.365242e+04
Unemployment,6.431219e+04
Store_11.0,8.535333e+04


In [230]:
fig = px.bar(feature_importance, orientation = 'h')
fig.update_layout(showlegend = False, 
                  margin = {'l': 120} 
                 )
fig.show()

On observe que : 
- Les stores 5, 7 et 9 enregistrent les meilleures ventes hebdomadaires. 
- "CPI", "Unemployment" et "Temperature" sont les varaibles les plus importantes dans la détermination des ventes hebdomadaires. 

##### Regularization

In [231]:
print("3-fold cross-validation...")
model = Ridge()
scores = cross_val_score(model, X_train, Y_train, cv=3)
print('The cross-validated R2-score is : ', scores.mean())
print('The standard deviation is : ', scores.std())

3-fold cross-validation...
The cross-validated R2-score is :  0.770919921761808
The standard deviation is :  0.04791996940533174


##### Grid search

In [232]:
print("Grid search...")
model = Ridge()
params = {
    'alpha': [0.0, 0.1, 0.5, 1.0]
}
gridsearch = GridSearchCV(model, param_grid = params, cv = 3) # cv : the number of folds to be used for CV
gridsearch.fit(X_train, Y_train)
print("...Done.")
print("Best hyperparameters : ", gridsearch.best_params_)
print("Best R2 score : ", gridsearch.best_score_)

Grid search...
...Done.
Best hyperparameters :  {'alpha': 0.1}
Best R2 score :  0.8798354275795384


##### Evaluation

In [233]:
print("R2 score on training set : ", gridsearch.score(X_train, Y_train))
print("R2 score on test set : ", gridsearch.score(X_test, Y_test))

R2 score on training set :  0.9753018975069008
R2 score on test set :  0.9302883236535135


#### Conclusion : Les performances du modèle de régression linéaire sont médiocres avec une seule variable explicative. Avec plusieurs variables explicatives, les performances deviennent très bonnes, et même encore meilleures en utilisant un modèle Ridge et une Grid Search. 